# Multiple Agents Interacting

In [1]:
import llama_cpp
import json

from IPython.display import clear_output

In [2]:
GLOBAL_LLM_INSTANCE = None # Run once per restart

In [73]:
# NOTE: these can be agent-specific if we want
TEMP = 0.7
REPEAT_PENALTY = 1.05

In [74]:
# NOTE: to use other underlying models in the future: we can make this an abstract class and subclass for each model type

class LLM:
    def __init__(self, system_prompt:str=''):
        global GLOBAL_LLM_INSTANCE
        """
        Create or use a global LLM instance, and initialize history

        Parameters
        ----------
        system_prompt:str - <see LLM.reset>
        """
        if GLOBAL_LLM_INSTANCE == None:
            GLOBAL_LLM_INSTANCE = llama_cpp.Llama(model_path='/data/ai_club/llms/llama-2-7b-chat.Q5_K_M.gguf', n_gpu_layers=-1, verbose=0, n_ctx=4000)
        self.reset(system_prompt)

    def __call__(self, prompt:str='', role:str='user', response_format:dict=None):
        """
        Elicit a response from the LLM

        Parameters
        ----------
        prompt:str - text to append to the history before eliciting a response, or an empty string to use the existing history without adding to it
        role:str - the role associated with the prompt text: 'user', 'system', or 'assistant'. Ignored if prompt is None.
        response_format:dict - a dict format to force the response to be in -- e.g., `{'to': '<who you are talking to>', 'response': '<your actual response>'}` -- or `None` for the response to be a regular string
        """

        if response_format:
            self._history += [{
                'role':'user',
                'content': 'Your next output should be formatted as this json with nothing extra: ' + json.dumps(response_format)
            }]

        if prompt:
            self._history += [{'role':role, 'content':prompt}]

        last_msg_idx = len(self._history)

        resp = self._force_chat_completion()
        resp_dict = None

        if response_format:
            while True:
                try:
                    if '}' not in resp:
                        resp += '}'
                    resp = resp[resp.index('{'):] # the json might be surounded by other text
                    resp_dict = json.loads(resp)
                    break
                except:
                    self._history += [{
                        'role':'user',
                        'content': 'Your previous output WAS NOT correctly formatted. Make sure it has necessary curly brackets and quotes. It shold be formatted as this json with nothing extra: ' + json.dumps(response_format)
                    }]
                    resp = self._force_chat_completion()

                # for debug:
                # clear_output(wait=True)
                # print(self.get_hist())
                print('bad json:', resp)

            # remove correction messages
            self._history = (
                self._history[0:last_msg_idx-2] + # up to format prompt
                self._history[last_msg_idx-1:last_msg_idx] + # user prompt
                self._history[-1:] # final response
            )

        return resp_dict if response_format else resp

    def _force_chat_completion(self):
        global GLOBAL_LLM_INSTANCE
        '''
        To fix bug where model response is blank.
        IMPORTANT: response is added to the history
        '''
        resp = None
        while resp == None or resp['content'] == '': 
            resp = GLOBAL_LLM_INSTANCE.create_chat_completion(self._history, temperature=TEMP, repeat_penalty=REPEAT_PENALTY)['choices'][0]['message']

        self._history += [resp]

        return resp['content']

    def get_hist(self) -> str:
        """
        Get a nicely-formatted string of the current history.
        """
        hist = ''
        for msg in self._history:
            hist += f'{msg["role"]} --- {msg["content"]}\n__________\n\n'
        return hist

    def reset(self, system_prompt:str=''):
        """
        Reset the LLM's chat history with a new system prompt.
        
        Parameters
        ----------
        system_prompt:str - instructions for the LLM, or an empty string to start without a system prompt
        """
        if system_prompt:
            self._history = [{'role':'system', 'content':system_prompt}]

In [79]:
# text colors
BLK = "\x1b[30m"
RED = "\x1b[31m"
GRN = "\x1b[32m"
YEL = "\x1b[33m"
BLU = "\x1b[34m"
MAG = "\x1b[35m"
CYN = "\x1b[36m"
WHT = "\x1b[37m"
RESET = "\x1b[0m"

name_colors = {
    'Jerome': YEL,
    'Bo': GRN,
    'Tom': CYN
}

agent_info = {
    'Jerome': 'a mighty barbarian',
    'Bo': 'a high-class frenchman who does not know any French',
    'Tom': 'an argumentative and highly opinionated tech entrepreneur'
}
agents = dict()
names_filtered = lambda ex: [n for n in agent_info.keys() if n != ex]

for name, status in agent_info.items():
    others = ', '.join(names_filtered(name))
    agents[name] = LLM(f'You are named {name}. You are {status}. The user is speaking to you on behalf of multiple people ({others}). Be concise, and do not duplicate responses.')

In [80]:
last_msg = {'from': '', 'to': 'Bo', 'msg': ''}

for _ in range(100):
    preface = ''
    if last_msg['from']:
        preface = last_msg['from'] + ' tells you: '
    resp = agents[last_msg['to']](
        preface + last_msg['msg'],
        response_format={'to':f"one of {', '.join(names_filtered(last_msg['to']))}", 'response':'your actual response'}
    )
    last_msg['from'] = last_msg['to']
    last_msg['to'] = resp['to']
    last_msg['msg'] = resp['response']

    from_, to = last_msg['from'], last_msg['to']

    print(f"{name_colors[from_]}{from_}{RESET} -> {name_colors[to]}{to}{RESET}\t::\t{last_msg['msg']}")

Bo -> Jerome	::	Bonjour Jerome! *adjusts monocle* How may I assist you today?
Jerome -> Bo	::	Greetings, noble Bo! *grins* It is an honor to serve you. How may I aid you in this fine day?
Bo -> Tom	::	Bonjour Tom! *adjusts monocle* I see you're looking sharp today. What can I help you with?
Tom -> Bo	::	Ah, Bonjour to you as well, old chap! *adjusts own monocle* I'm afraid I can't be bothered with your trivial inquiries, what with my cutting-edge tech startup to attend to. *rolls eyes* Do tell, what brings you to my doorstep today?
Bo -> Jerome	::	Ah, Tom old bean! *chuckles* I see you're as sharp as ever. Perhaps you could spare a moment to discuss the finer points of high society etiquette? *winks*
Jerome -> Tom	::	Hah! *raises an eyebrow* You want to talk etiquette with me, Tom? *chuckles* I'm afraid you're barking up the wrong tree. I'm a barbarian, not some stuffy noble. But hey, I'll listen if you've got some good stories to share.
Tom -> Jerome	::	Oh, I see. Well, I suppose you'

KeyboardInterrupt: 